In [ ]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import tensorflow.compat.v1 as tf

tf.logging.set_verbosity(tf.logging.ERROR)  # Hide TF deprecation messages
import numpy as np
import pickle
import modules
import data_utils

In [ ]:
chair_meshes_paths = list(glob("chairs_ngon/*.obj"))
with open("chairs_split_dict.pickle", 'rb') as f:
    chairs_split_dict = pickle.load(f)
train_paths = []
val_paths = []
test_paths = []
for c in chair_meshes_paths:
    try:
        split = chairs_split_dict[c.split("/")[-1].replace(".obj", "")]
    except KeyError:
        continue
    if split =='train':
        train_paths.append(c)
    elif split =='val':
        val_paths.append(c)
    else:
        test_paths.append(c)

In [ ]:
max_length = 30

In [ ]:
captions = pd.read_csv("captions_tablechair.csv").dropna()

In [ ]:
with open('chair_to_text_full.pickle', 'rb') as f:
    chair_to_text = pickle.load(f)
with open('word_to_int_full.pickle', 'rb') as f:
    word_to_int = pickle.load(f)
with open('int_to_word_full.pickle', 'rb') as f:
    int_to_word = pickle.load(f)

In [ ]:
def text2shape(paths):
    for path in paths:
        mesh_dict = data_utils.load_process_mesh(path)
        if len(mesh_dict['vertices'])>800:
            continue
        if len(mesh_dict['faces'])>2800:
            continue
        try:
            texts =  chair_to_text[path.split("/")[-1].replace(".obj", "")]
        except KeyError:
            continue
        text = texts[0][:max_length]
        mesh_dict['text_feature'] = np.pad(text, (0,max_length-len(text)))
        yield mesh_dict

In [ ]:
Text2ShapeDataset = tf.data.Dataset.from_generator(
        lambda:text2shape(test_paths),
        output_types={
            'vertices': tf.int32, 'faces': tf.int32,
            'text_feature': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
            'text_feature':tf.TensorShape(max_length)})
vertex_model_dataset = data_utils.make_vertex_model_dataset(Text2ShapeDataset, apply_random_shift=False)
vertex_model_dataset = vertex_model_dataset.repeat()
vertex_model_dataset = vertex_model_dataset.padded_batch(16, padded_shapes=vertex_model_dataset.output_shapes)
vertex_model_dataset = vertex_model_dataset.prefetch(1)

In [ ]:
vertex_model = modules.TextToVertexModel(
    decoder_config=dict(
      hidden_size=128,
      fc_size=512,
      num_heads=8,
      layer_norm=True,
      num_layers=24,
      dropout_rate=0.4,
      re_zero=True,
      memory_efficient=True
      ),
    path_to_embeddings="glove/glove.6B.100d.txt",
    embedding_dims = 100,
    quantization_bits=8,
    vocab = word_to_int,
    max_num_input_verts=800,  # number of vertices in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    use_discrete_embeddings=True
)

In [ ]:
o = (i for i in text2shape(test_paths))
SIZE = sum(1 for _ in o)

In [ ]:
it = vertex_model_dataset.make_initializable_iterator()
vertex_model_batch = it.get_next()
iterator_init_op = it.initializer
vertex_model_pred_dist = vertex_model(vertex_model_batch)
vertex_model_loss = -tf.reduce_sum(
    vertex_model_pred_dist.log_prob(vertex_model_batch['vertices_flat']) *
    vertex_model_batch['vertices_flat_mask'])
saver_vertex = tf.train.Saver(var_list=vertex_model.variables)
with tf.Session() as sess:
    saver_vertex.restore(sess, "text_vertex_shapeglot/best-237")
    sess.run(iterator_init_op)
    mean_loss = []
    bits_per_vertex = []
    for _ in tqdm(range(int(np.ceil(SIZE/16)))):
        batch, loss =  sess.run((vertex_model_batch, vertex_model_loss))
        mean_loss.append(loss/16)
        num_vertices = 0
        for b in range(batch['vertices'].shape[0]):
            k = batch['vertices'][b]
            k = k[~np.all(k == 0, axis=1)]
            num_vertices+=k.shape[0]
        bits_per_vertex.append(loss /  (np.log(2)*num_vertices))
    print(np.array(mean_loss).mean())
    print(np.array(bits_per_vertex).mean())

In [ ]:
vertex_samples_val = vertex_model.sample(
    16, context=vertex_model_batch, max_sample_length=800, top_p=0.95,
    recenter_verts=False, only_return_complete=False)
with tf.Session() as sess:
    saver_vertex.restore(sess, "text_vertex_shapeglot/best-237")
    sess.run(iterator_init_op)
    acc = []
    for _ in tqdm(range(int(np.ceil(SIZE/16)))):
        gt, pred = sess.run((vertex_model_batch, vertex_samples_val))
        gt = gt['vertices']
        pred = pred['vertices']*np.dstack([pred['vertices_mask'], pred['vertices_mask'], pred['vertices_mask']])
        for b in range(pred.shape[0]):
            k = gt[b]
            k = data_utils.dequantize_verts(k[~np.all(k == 0, axis=1)])
            gt_padded = np.pad(k, ((0, 800-k.shape[0]), (0, 0)))
            acc.append((gt_padded == pred[b]).sum()/len(gt_padded.flatten()))
    print(np.array(acc).mean())